## Imports

In [60]:
import time
from datetime import datetime
import logging
import pytz
import requests
from pymongo import MongoClient
import config
import pandas as pd
import json
import copy
import plotly.offline as py
import plotly.graph_objs as go

FORMAT = "%(levelname)s - %(asctime)s - %(message)s"
logging.basicConfig(
        filename='prototype_recording_util.log',
        filemode='w',
        format=FORMAT,
        level=logging.ERROR)
LOGGER = logging.getLogger()

In [39]:
LOCATIONS = {
    'Bay Area': {
        'work': {
            'CUP': {
                'name': 'Cupertino, MA',
                'lat_lon': '37.330227595678146,-122.03281591229046'
            }
        },
        'home': {
            'SCZ': {
                'name': 'Santa Cruz, Downtown',
                'lat_lon': '36.974797779340655,-122.0290861946705'
            },
            'OAK': {
                'name': 'Oakland, West',
                'lat_lon': '37.812088988762454,-122.29942506950347'
            },
            'SJC': {
                'name': 'Santa Jose, Downtown',
                'lat_lon': '37.33481534485126,-121.88822099401446'
            },
            'SFP': {
                'name': 'San Francisco, Pacific Heights',
                'lat_lon': '37.79427344559132,-122.43493160658136'
            },
            'LGS': {
                'name': 'Los Gatos, Downtown',
                'lat_lon': '37.22473891235269,-121.98375589032867'
            },
            'RWC': {
                'name': 'Redwood City, Oxford',
                'lat_lon': '37.46947269135535,-122.2284655255091'
            },
            'PCA': {
                'name': 'Pacifica, Linda Mar',
                'lat_lon': '37.594122784571354,-122.5015289019967'
            },
            # 'CAM': {
            #     'name': 'Campbell, Downtown',
            #     'lat_lon': '37.28750197556045,-121.94487365182788'
            # },
            # 'MLP': {
            #     'name': 'Menlo Park, West MP',
            #     'lat_lon': '37.431029416205284,-122.20217060835637'
            # },
            # 'CPA': {
            #     'name': 'Capitola, Mall',
            #     'lat_lon': '36.97371780004841,-121.95931307716036'
            # },
            # 'BER': {
            #     'name': 'Berkeley, Downtown',
            #     'lat_lon': '37.87075376688075,-122.27199562287598'
            # },
            # 'SCU': {
            #     'name': 'Santa Clara, SCU',
            #     'lat_lon': '37.34887015070597,-121.94401506069336'
            # },
            # 'FEL': {
            #     'name': 'Felton, Downtown',
            #     'lat_lon': '37.05131035573837,-122.07406272553854'
            # },
            # 'WLG': {
            #     'name': 'Willow Glen, Doerr Park',
            #     'lat_lon': '37.2739703906996,-121.91368527984521'
            # },
            # 'SFM': {
            #     'name': 'San Francisco, Mission District',
            #     'lat_lon': '37.75870633260038,-122.41794365829263'
            # },
            # 'MTV': {
            #     'name': 'Mountain View, Shoreline',
            #     'lat_lon': '37.403712687363814,-122.07814772790742'
            # },
        }
    },
    'San Diego': {
        'work': {
            'RBO': {
                'name': 'Rancho Bernardo, VF',
                'lat_lon': '33.01199872926068,-117.09188697836275'
            }
        },
        'home': {
            'SAN': {
                'name': 'San Diego Airport, T1 Ride Share',
                'lat_lon': '32.73183061880728,-117.19766417435267'
            },
            'CLM': {
                'name': 'Clairemont, Town Square',
                'lat_lon': '32.8265288909409,-117.20598237549127'
            },
            'CBD': {
                'name': 'Carlsbad, Village',
                'lat_lon': '33.160773694686696,-117.34669932279765'
            },
            'NPK': {
                'name': 'North Park',
                'lat_lon': '32.750866591339886,-117.13018958965476'
            },
            'CDO': {
                'name': 'Casa de Oro-Mount Helix',
                'lat_lon': '32.767445120760335,-116.97276250515775'
            },
            'DMH': {
                'name': 'Del Mar Heights',
                'lat_lon': '32.95442922821245,-117.23444297327268'
            },
            'RPO': {
                'name': 'Rancho Ponderosa',
                'lat_lon': '33.070781939107064,-117.24470653655378'
            },
            'FBK': {
                'name': 'Fallbrook, South',
                'lat_lon': '33.317818161062206,-117.21989626093905'
            }
        }
    }
}

# Invert locations into a map of city code: locality
locality_map = {}
for locality_key, locality_value in LOCATIONS.items():
    for city_type_value in locality_value.values():
        for city in city_type_value.keys():
            locality_map[city] = locality_key
locality_map

{'CUP': 'Bay Area',
 'SCZ': 'Bay Area',
 'OAK': 'Bay Area',
 'SJC': 'Bay Area',
 'SFP': 'Bay Area',
 'LGS': 'Bay Area',
 'RWC': 'Bay Area',
 'PCA': 'Bay Area',
 'RBO': 'San Diego',
 'SAN': 'San Diego',
 'CLM': 'San Diego',
 'CBD': 'San Diego',
 'NPK': 'San Diego',
 'CDO': 'San Diego',
 'DMH': 'San Diego',
 'RPO': 'San Diego',
 'FBK': 'San Diego'}

### Import Offline Commutes

In [40]:
with open ('commutes.json') as file:
    commutes = json.load(file)

In [67]:
df = pd.DataFrame.from_dict(commutes['commutes'])

# Cast datetime strings to datetimes
df['departureTime'] = pd.to_datetime(df['departureTime'])
# Convert timestamp from UTC to local
df['departureTimeLocal'] = df['departureTime'].dt.tz_convert(tz='America/Los_Angeles')
# Extract the local weekday (Monday == 0, Sunday == 6), hour, and minute
df['departureWeekdayLocal'] = df['departureTimeLocal'].dt.weekday
df['departureHourLocal'] = df['departureTimeLocal'].dt.hour
df['departureMinuteLocal'] = df['departureTimeLocal'].dt.minute

# Assign locality to rows that don't have one
df['locality'] = df['origin'].map(locality_map)
# Drop rows where the locality is still NaN
# This implies that the city pair is not actively being recorded
df = df.dropna(subset = ['locality'])

df = df.drop(['_id', 'departureTimeLocalizedSimplified'], axis='columns')

df

,origin,destination,departureTime,travelTimeInSeconds,locality,departureTimeLocal,departureWeekdayLocal,departureHourLocal,departureMinuteLocal
0,SCZ,CUP,2021-12-14 00:30:00+00:00,2350,Bay Area,2021-12-13 16:30:00-08:00,0,16,30
1,CUP,SCZ,2021-12-14 00:30:00+00:00,2514,Bay Area,2021-12-13 16:30:00-08:00,0,16,30
3,CUP,CAM,2021-12-14 00:30:00+00:00,847,Bay Area,2021-12-13 16:30:00-08:00,0,16,30
5,CUP,SCU,2021-12-14 00:30:00+00:00,894,Bay Area,2021-12-13 16:30:00-08:00,0,16,30
6,SJC,CUP,2021-12-14 00:30:00+00:00,833,Bay Area,2021-12-13 16:30:00-08:00,0,16,30
...,...,...,...,...,...,...,...,...,...
1465474,RBO,DMH,2023-09-16 21:45:00+00:00,1089,San Diego,2023-09-16 14:45:00-07:00,5,14,45
1465475,RPO,RBO,2023-09-16 21:45:00+00:00,1820,San Diego,2023-09-16 14:45:00-07:00,5,14,45
1465476,RBO,RPO,2023-09-16 21:45:00+00:00,1884,San Diego,2023-09-16 14:45:00-07:00,5,14,45
1465477,FBK,RBO,2023-09-16 21:45:00+00:00,1764,San Diego,2023-09-16 14:45:00-07:00,5,14,45


In [149]:
cities = set(df['origin'].to_list())

fig = go.Figure()

for origin in ['SCZ']:
    other_cities = cities.copy()
    other_cities.remove(origin)
    
    for destination in ['CUP']:
        route_df = df[
            (df['origin'] == origin)
            & (df['destination'] == destination)
        ]

        # for weekday in sorted(set(route_df['departureWeekdayLocal'].to_list())):
        for weekday in [1]:
            # for hour in sorted(set(route_df['departureHourLocal'].to_list())):
            for hour in [8]:
                # for minute in sorted(set(route_df['departureMinuteLocal'].to_list())):
                for minute in [0]:
                    route_at_time_df = route_df[
                        (route_df['departureWeekdayLocal'] == weekday)
                        & (route_df['departureHourLocal'] == hour)
                        & (route_df['departureMinuteLocal'] == minute)
                    ]

                    fig.add_trace(go.Scattergl( 
                        x=route_at_time_df['departureTimeLocal'],
                        y=round((route_at_time_df['travelTimeInSeconds'] / 60).rolling(window=12).mean(), 1),
                        name=f'{origin}-{destination} {weekday} {hour}:{minute}',
                        mode='lines+markers'
                    ))

fig.update_layout(
    showlegend=True,
    xaxis_title='Departure Date',
    yaxis_title='Travel Time (min)'
)
py.iplot(fig, filename=f'{origin}_{destination}_{weekday}_{hour}_{minute}')

In [133]:
city_pair = ('SCZ', 'CUP')


fig = go.Figure()
for direction in ['forward', 'reverse']:
    if direction == 'forward':
        origin = city_pair[0]
        destination = city_pair[1]
    else:
        origin = city_pair[1]
        destination = city_pair[0]

    route_df = df[
        (df['origin'] == origin)
        & (df['destination'] == destination)
    ]

    fig.add_trace(go.Scattergl( 
        x=route_df['departureTimeLocal'],
        y=round(route_df['travelTimeInSeconds'] / 60),
        name=f'{origin}-{destination}',
        mode='lines+markers'
    ))

fig.update_layout(
    showlegend=True,
    xaxis_title='Departure Date/Time',
    yaxis_title='Travel Time (min)'
)
py.iplot(fig, filename=f'{origin}_{destination}')